In [1]:
import psycopg2 as ppg
import pandas as pd
import numpy as np
import datetime as ct
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import pickle
import math
import matplotlib.pyplot as plt
import time
import datetime 

In [2]:
creds = """dbname=dublin_bus_db user=user1 password=manyCricketsChirp 
host=csi420-01-vm5.ucd.ie port = 80"""

In [3]:
conn = ppg.connect(creds)
cur = conn.cursor()

In [4]:
SQL = """
SELECT "ActualTime_Arr", "DayOfService", "PlannedTime_Arr","PlannedTime_Dep","LineID"
FROM public.rt_trips_table ORDER BY "DayOfService" ASC
"""
# WHERE "LineID" = '46A' 
# AND "Direction" = 1
# AND "ActualTime_Arr" is not null
# AND "ActualTime_Dep" is not null;

In [5]:
cur.execute(SQL)

In [6]:
rows = cur.fetchall()
cols = np.array(["ActualTime_Arr", "DayOfService", "PlannedTime_Arr","PlannedTime_Dep","LineID"])


In [7]:
cur.close()
conn.close()

In [8]:
df = pd.DataFrame(rows, columns = cols)
df.head()

,ActualTime_Arr,DayOfService,PlannedTime_Arr,PlannedTime_Dep,LineID
0,38022,2018-01-01,38417,36000,1
1,49653,2018-01-01,50199,47400,1
2,61566,2018-01-01,61178,58200,1
3,64454,2018-01-01,64486,61800,1
4,77498,2018-01-01,77827,75600,1


In [9]:
df.dtypes


ActualTime_Arr     object
DayOfService       object
PlannedTime_Arr    object
PlannedTime_Dep    object
LineID             object
dtype: object

In [10]:
df["DayOfService"]=df["DayOfService"].apply(lambda x: x.strftime('%d%m%Y'))


In [11]:
df['DAY']=df['DayOfService'].str[0:2]
df['MON']=df['DayOfService'].str[2:4]
df['YEAR']=df['DayOfService'].str[4:]
df

,ActualTime_Arr,DayOfService,PlannedTime_Arr,PlannedTime_Dep,LineID,DAY,MON,YEAR
0,38022,01012018,38417,36000,1,01,01,2018
1,49653,01012018,50199,47400,1,01,01,2018
2,61566,01012018,61178,58200,1,01,01,2018
3,64454,01012018,64486,61800,1,01,01,2018
4,77498,01012018,77827,75600,1,01,01,2018
5,41873,01012018,42164,39600,1,01,01,2018
6,71732,01012018,71388,69000,1,01,01,2018
7,79237,01012018,79747,77400,1,01,01,2018
8,43724,01012018,43817,41400,1,01,01,2018
9,56527,01012018,56235,53400,1,01,01,2018


In [ ]:


# df1=pd.DataFrame({"ActualTime_Arr", "DayOfService", "PlannedTime_Arr","PlannedTime_Dep","LineID","PlannedTime_Dep": df['PlannedTime_Dep'][:1000], "Time":0,"DayOfService":df['DayOfService'][:1000]})
# df1["DayOfService"]=df1["DayOfService"].apply(lambda x: x.strftime('%d%m%Y'))



In [12]:


df["PlannedTime_Dep"] = df["PlannedTime_Dep"].astype('int64')
df.dtypes


ActualTime_Arr     object
DayOfService       object
PlannedTime_Arr    object
PlannedTime_Dep     int64
LineID             object
DAY                object
MON                object
YEAR               object
dtype: object

In [ ]:
for i,val in df.iterrows():
#      print(val['PlannedTime_Dep'])
    value=time.strftime('%H:%M',time.gmtime(val['PlannedTime_Dep']))
    val["Planned_Time"]=value
    
#     myval=time.strftime('%H%M%S',time.gmtime(val))
# print(type(myval))
#      if myval.isdigit():
#         value=int(myval)
#         print(i,value)
#     else:
#         print(i,myval)
            
#             =int(time.strftime('%H%M%S', time.gmtime(val)))

In [ ]:
# df1['DAY']=df1['DayOfService'].str[0:2]
# df1['MON']=df1['DayOfService'].str[2:4]
# df1['YEAR']=df1['DayOfService'].str[4:]

In [ ]:
df1=df1.drop(["DayOfService","PlannedTime_Dep"],axis=1)

In [ ]:
df1

# Weather Data import

In [ ]:
temp_df = pd.read_csv('hly175.csv',skiprows=14)

In [ ]:

temp_df=temp_df.drop(temp_df.index[0:98985])
temp_df=temp_df.drop(["ind","ind.1","wetb","dewpt","vappr","rhum","msl","ind.2"],axis=1)
temp_df


In [ ]:
df1

In [ ]:
temp_df['Time']= temp_df["date"].str[11:]
temp_df['DAY']=temp_df["date"].str[:2]
temp_df['MON']=temp_df["date"].str[3:6]
temp_df['YEAR']=temp_df["date"].str[7:11]
mapit= {'jan':'01', 'feb':'02', 'mar':'03', 'apr':'04','may':'05', 'jun':'06','jul':'07','aug':'08','sep':'09','oct':'10','nov':'11','dec':'12'}

In [ ]:
temp_df.MON= temp_df.MON.map(mapit)
temp_df=temp_df.drop(["date"],axis=1)

In [ ]:
temp_df

In [ ]:
for idx,row in temp_df.iterrows():
      if row['']

In [ ]:
DF2=pd.merge(df1,temp_df, on=['DAY','MON','YEAR'])

In [ ]:
mdf=pd.merge_asof(temp_df,df1,on=['Time','DAY','MON','YEAR'] )

In [ ]:
DF2

In [ ]:
# temp_df=temp_df.set_index('Time')
# temp_df

In [ ]:
DF2=pd.merge(df1,temp_df, on=['Time','DAY','MON','YEAR'])

In [ ]:
DF2

In [ ]:
temp_df['new_date']

In [ ]:
temp_df

In [ ]:
temp_df.iloc[1, 1:] = df2.iloc[1, 1:].apply(pd.to_datetime, format = '%d-%b-%Y')

In [ ]:
df.tail()

In [ ]:
df.describe().T

In [ ]:
df.dtypes

In [ ]:
df["ActualTime_Arr"] = df["ActualTime_Arr"].astype("int64")
df["PlannedTime_Arr"] = df["PlannedTime_Arr"].astype("int64")
df["PlannedTime_Dep"] = df["PlannedTime_Dep"].astype("int64")

In [ ]:
df.dtypes

In [ ]:
df.head()

In [ ]:
a = np.array(df["ActualTime_Arr"])

In [ ]:
# Convert the timedelta in seconds to a time
# This does not need to be done for the model but
# must be converted for the user
# converter = lambda delta: str(ct.timedelta(seconds = delta))
# b = np.array([converter(int(xi)) for xi in a])
# b

In [ ]:
# str(ct.timedelta(seconds=666))

In [ ]:
procdf = df.copy()

In [ ]:
# procdf["ActualTime_Arr"] = b

In [ ]:
procdf.head(5)

Discussion
=========
The actual arrival time of a bus at the end of the route is functionally dependent upon the prevailing conditions on the route, which include the day, month, planned_dep_time and planned_arr_time. These latter two are important because they operate as time coordinates that uninquely identify the trip. When people plan trips they typically consider the departure time and arrival time, and select the departure time that gets them to their destination within a resasonable margin of the arrival time. For instance if a departure time leaves only 5 minutes margin between the planned arrival time and the time the person needs to get to a venue, they may choose an earlier departure time. The predicted time to arrival will be learned from 

Next need to extract month and day from the date. This will allow the machine model to accept month and day as arguments, after it has been trained on historical weather data that is merged into the dataset according to the date. The purpose for the full date is to allow for the weather data to be added and the month and day to be extracted. 

ActualTime_Dep/Arr can't be used for forecasting arrival in the future because the trip hasn't happened yet, and the data input to the predictive model has to be the same tuple of data as it was trained with, although with unseen values. 

One possible reason others haven't considered weather is because weather forecasts are typically only available a week in advance and so the predictive model would be limited by the forecast range available from the weather data API. So it may be sensible to have two different models: one for when weather is available for the given travel dates and the other for when weather isn't available. 

In [ ]:
dates = procdf["DayOfService"]
t = str(dates[0]).split('-')
y = t[0]
m = t[1]
d = t[2]
type(dates)

In [ ]:
dates[0].weekday()

In [ ]:
def decompt(dates: pd.core.series.Series)->np.array:
    days = {0: "Mon", 1: "Tues", 2: "Wed", 3: "Thurs", 4: "Fri", 5: "Sat", 6: "Sun"}
    y = list(range(len(dates)))
    m = list(range(len(dates)))
    d = list(range(len(dates)))
    for i in range (len(dates)):
        wkday = dates[i].weekday()
        t = str(dates[i]).split("-")
        y[i] = t[0]
        m[i] = t[1]
#         d[i] = days[wkday] can't use strings in linear regresson mode
        d[i] = wkday
    return np.array(y), np.array(m), np.array(d)

In [ ]:
ts = decompt(dates) # takes about 15sec to run; linear complexity

In [ ]:
ts

In [ ]:
procdf['month'] = ts[1]
procdf['day'] = ts[2]

In [ ]:
procdf.head()

In [ ]:
cols = procdf.columns.tolist()
cols = [cols[0], cols[1], cols[4], cols[5], cols[2], cols[3]]
cols

In [ ]:
procdf2 = procdf[cols]
procdf2.tail()

In [ ]:
procdf2.dtypes

In [ ]:
# Remove the DayOfService Column
objdf = procdf2.copy()
objdf = pd.DataFrame(procdf2[["ActualTime_Arr", "month", 
                              "day", "PlannedTime_Arr", "PlannedTime_Dep"]])

In [ ]:
objdf.head()

In [ ]:
l = list(objdf["month"].unique())

In [ ]:
# all months are accounted for
l.sort()
l

In [ ]:
print("Number of duplicateed rows: ", objdf.duplicated().sum())

In [ ]:
print("Number of duplicateed columns: ", objdf.T.duplicated().sum())

In [ ]:
# show duplicates 
objdf['is_duplicated'] = objdf.duplicated()
dups = objdf.index[objdf['is_duplicated'] == 1].tolist() # get index of duplicates
print(dups)
print(objdf.shape)

In [ ]:
# drop duplicates by index
objdf = objdf.drop(dups)

In [ ]:
objdf.shape

In [ ]:
# Drop is_duplicated column
objdf = objdf.drop(['is_duplicated'], axis=1)

Train a Linear Regression Model
=========================


In [ ]:
# Split for training and testing
dfTrain, dfTest = train_test_split(objdf, shuffle=True, test_size=0.333)

In [ ]:
# Reshuffle and Reindex the training sample
dfTrain = dfTrain.sample(frac=1).reset_index(drop=True)
print(dfTrain.shape)
dfTrain.head()

In [ ]:
dfTest = dfTest.sample(frac=1).reset_index(drop=True)
print(dfTest.shape)
dfTest.head()

In [ ]:
# Save the dataframes for later use
# dfTrain.to_csv('dfTrain.csv')
# dfTest.to_csv('dfTest.csv')

In [ ]:
# Target vector
y = dfTrain['ActualTime_Arr']
y.values

In [ ]:
# Sample matrix
x_features = ["month", "day", "PlannedTime_Arr", "PlannedTime_Dep"]
X = dfTrain[x_features]
X.values

In [ ]:
lrm = LinearRegression().fit(X, y)

In [ ]:
print("Feature: \n", "ActualTime_Arr")
print("Coeficient/Weight: \n", lrm.coef_)
print("Intercept/Bias: \n", lrm.intercept_)

Linear Model Learned
=================
<p><b>ActualTime_Arr</b> = 50.6752754718691 + [(10.6109851 * <b>month</b>) + (22.154156 * <b>day</b>) + (1.02015296 * <b>PlannedTime_Arr</b>) + (-0.0197502832 * <b>PlannedTime_Dep</b>)] </p>

In [ ]:
# save the model
# pickle.dump(lrm, open("rough_lrm.sav", "wb"))

In [ ]:
model = pickle.load(open("rough_lrm.sav", "rb"))
model.score(X, y) # coefficient of determination = R^2: 1.0 is best possible score

In [ ]:
predicts = lrm.predict(dfTrain[x_features])
d = np.array(predicts)
b = pd.DataFrame(d, columns=["Predicted_ActualTime_Arr"])
b = b.join(dfTrain["ActualTime_Arr"])
b.head()

In [ ]:
b.loc[b.index.values[1]].diff()[1]

In [ ]:
sum = []
sum_2 = []
for index in b.index.values:
    sum.append(b.loc[index].diff()[1])
    sum_2.append(b.loc[index].diff()[1]**2) # append the square of the difference
s = pd.Series(sum_2)
s = math.sqrt(s.sum()/b.shape[0]) # Root mean square 
s/60 # error in minutes

In [ ]:
sumdfTrain = pd.DataFrame(sum)
sumdfTrain.describe().T

In [ ]:
plt.figure(dpi=600)
num_bins = 15
plt.xlabel('Error [s]')
plt.ylabel('log#Errors')
plt.title('Training Error Distribution')
plt.hist(sum, num_bins, facecolor = 'red', log=True)
# plt.savefig('dist_training_errors', bbox_inches = 'tight')
plt.show()

### Test the model on unseen dfTest

In [ ]:
predictions = lrm.predict(dfTest[x_features])

In [ ]:
q = np.array(predictions)

In [ ]:
p = pd.DataFrame(q, columns=["Predicted_ActualTime_Arr"])
p = p.join(dfTest["ActualTime_Arr"])
p.head()

In [ ]:
# Get differences betweeen actual and predicted value
p.loc[p.index.values[1]].diff()[1]

In [ ]:
sum = []
sum_2 = []
for index in p.index.values:
    sum.append(p.loc[index].diff()[1])
    sum_2.append(p.loc[index].diff()[1]**2) # append the square of the difference
s = pd.Series(sum_2)
s = math.sqrt(s.sum()/p.shape[0]) # Root mean square 
s/60 # error in minutes

In [ ]:
sumdfTrain = pd.DataFrame(sum)
sumdfTrain.describe().T

In [ ]:
plt.figure(dpi=600)
num_bins = 15
plt.xlabel('Error [s]')
plt.ylabel('log#Errors')
plt.title('Test Error distribution')
plt.hist(sum, num_bins, facecolor = 'red', log=True)
# plt.savefig('dist_test_errors', bbox_inches = 'tight')
plt.show()

Discussion
=========
The **lrm** **_ActualTime__Arr** prediction deviates from the actual value by about 9 minutes with respect to both the training data and the test data. Compare this deviation with the Planned arrival time to see if the **lrm** is better than the method being used to make the schdeule. 

Take the **_PlannedTime__Arr** from just the test data and calculate the MSE between that and **_ActualTIme__Arr**. We can then see if our prediction is better than their prediction.

The performance of any model may be improved by weather data. So far the model has been resolving by month and day, but the weather conditions on any given day or month may be a significant factor in the error. **Month** encodes prevailing weather information implicitly but **day** does not given that weather may change drastically between any pair of given 'Mondays', for instance. What's more, the light level is significant (need citation) and **month** implicity encodes light-level information. A better model therefore would include explicit prevailing weather and light-level information. 

One suspects that varying light-level could be more impactful moving distally from the equator. Equatorial light levels do not vary significantly throughout the year relative to more polar regions, except through the day/night cycle. At the equator, there is no significant seasonal cycling. Seasonality is ostensibly a super-tropical phenomenon.